# Data Pre-Processing Phase 2 - Deriving new relations

In [1]:
import json
import itertools
import math


import pandas as pd
import datetime
from collections import Counter, defaultdict, deque
from pathlib import Path
from tqdm import tqdm
import random
import time
import tracemalloc
from tqdm import tqdm
import pytz  # type: ignore
from tweepy import User
from shutil import copy2

In [2]:
# Input data
DATA_DIR = Path('/data/TwiBot-22')
USER_DIR = DATA_DIR / 'processed' / 'users'
RELATION_DIR = DATA_DIR / 'processed' / 'relations'

# Output directory
OUT_DIR = DATA_DIR / 'processed' / 'derived'
OUT_DIR.mkdir(exist_ok=True)

In [3]:
users = [user for user in USER_DIR.glob("*")]
usernames = set([user.stem for user in users])

# Retweeted

We already have the `retweeted` relation, however, we need to transform it from tweet entities to user entities.
We start out with a mapping of the following form:

| source_id | relation | target_id |
|----------------------|-----------|---------------------|
| t1250366502454296576 | retweeted | t1250345547874979842
| t1488734114794532870 | retweeted | t1488722567741464579
| t1431316326992146438 | retweeted | t1430918145397645315

Then we merge this DataFrame twice, we use the `post` relation to get the users for the corresponding tweets.

| source_id | relation | target_id |
|------------|------|--------------------|
| u124137240 | post | t1497615862374998018 |
| u56901370 | post | t1487344639556210689 |
| u2943060805 | post | t1498507430187122688 |


As a result, we get two DataFrames formatted like this:

| source_id_user | source_id_tweet | target_id_tweet |
| ---------------------|----------------------|----------------------|
| u1076092249279217664 | t1250366502454296576 | t1250345547874979842 |
| u384126992 | t1488734114794532870 | t1488722567741464579 |
| u76896397 | t1431316326992146438 | t1430918145397645315 |


| source_id_tweet | target_id_tweet | target_id_user |
|----------------------|----------------------|----------------------|
| t1250366502454296576 | t1250345547874979842 | u1207540481997099009 |
| t1488734114794532870 | t1488722567741464579 | u933480950515863552 |
| t1488865835636641795 | t1488722567741464579 | u933480950515863552 |

Finally, we need to merge them and discard all the unnecessary columns.

| source_id   | relation  | target_id  |
|-------------|-----------|------------|
| u52664889   | retweeted_user | u868112707 |
| u337483640  | retweeted_user | u4795561   |
| u3301643341 | retweeted_user | u691353    |

In [4]:
retweeted_df = pd.read_csv(RELATION_DIR / 'retweeted.csv')

In [5]:
post_df = pd.read_csv(RELATION_DIR / 'post.csv')

### Retweeted Merge user for Source_id 

In [6]:
retweeted_user_left = pd.merge(retweeted_df[["source_id", "target_id"]], post_df[["source_id", "target_id"]], left_on='source_id', right_on='target_id', suffixes=('_retweeted', '_post'))

In [7]:
retweeted_user_left = retweeted_user_left[["source_id_post", "source_id_retweeted", "target_id_retweeted"]]
retweeted_user_left.columns = ["source_id_user", "source_id_tweet", "target_id_tweet"]

Lets make a quick sanity check that all our operation had the intended effect.
We want to make sure that all values are mapped correctly.

In [8]:
assert post_df.loc[post_df["target_id"] == retweeted_user_left.iloc[0]["source_id_tweet"]]["source_id"].item() == retweeted_user_left.iloc[0]["source_id_user"], "wrong mapping"

In [9]:
retweeted_user_left = retweeted_user_left.sort_values(by=["source_id_tweet", "target_id_tweet"])

### Retweeted Merge user for target_id

In [10]:
retweeted_user_right = pd.merge(retweeted_df[["source_id", "target_id"]], post_df[["source_id", "target_id"]], left_on='target_id', right_on='target_id', suffixes=('_retweeted', '_post'))

In [11]:
retweeted_user_right.columns = ["source_id_tweet", "target_id_tweet", "target_id_user"]

Sanity checks again, this time we also check that the shape of both dataframes is the same before we merge them.

In [12]:
assert post_df.loc[post_df["target_id"] == retweeted_user_right.iloc[0]["target_id_tweet"]]["source_id"].item() == retweeted_user_right.iloc[0]["target_id_user"], "Wrong mapping"
assert retweeted_user_left.shape == retweeted_user_right.shape, "Shapes do not match!"

In [13]:
retweeted_user = pd.merge(retweeted_user_left, retweeted_user_right, left_on=["source_id_tweet", "target_id_tweet"], right_on=["source_id_tweet", "target_id_tweet"]) #, suffixes=('_left', '_right'))

In [14]:
retweeted_user["relation"] = "retweeted_user"
retweeted_user = retweeted_user[["source_id_user", "relation", "target_id_user"]]
retweeted_user.columns = ["source_id", "relation", "target_id"]

In [15]:
assert retweeted_user.shape == retweeted_user_right.shape, "shape mismatch"

In [16]:
retweeted_user.to_csv(OUT_DIR / "retweeted_user.csv", index=False)

## New relation: Co-Retweeted
Now we aim to find users that retweeted the same tweet.
We reuse the `retweeted_user_left` DataFrame. So, we start out with:


| source_id_user | source_id_tweet | target_id_tweet |
| ---------------------|----------------------|----------------------|
| u1076092249279217664 | t1250366502454296576 | t1250345547874979842 |
| u384126992 | t1488734114794532870 | t1488722567741464579 |
| u76896397 | t1431316326992146438 | t1430918145397645315 |

Then, we group this DataFrame by the `target_id_tweet` column and discard the `source_id_tweet` column.
Now we have a list of users that retweeted a tweet for each entry in `target_id_tweet`.
For this relation we are only interested in tweets that get retweeted by more than one user, therefore, we discard all rows where the resulting list does not contain at least 2 users.
We end up with a DataFrame that looks this:

| target_id_tweet | source_id_user |
|----------------------|------------------------------------|
| t1000124286948990976 | [u2196348115, u1281526188] |
| t1000207766819098624 | [u712263962, u243036743] |
| t1000346560905887745 | [u2981968107, u15960453, u1150918] |

Finally, we take the lists inside `source_id_user` and create combinations of all users, excl. pairs with the same user twice.
These are now our edges.

| source_id | relation | target_id |
|---------------------|--------------|-------------|
| u28649627 | co_retweeted | u2462436602 |
| u243292912 | co_retweeted | u1700157950 |
| u960015338095239168 | co_retweeted | u1400517288 |

In [17]:
retweeted_grouped = retweeted_user_left.groupby("target_id_tweet")["source_id_user"].apply(list).reset_index()
retweeted_grouped = retweeted_grouped[retweeted_grouped['source_id_user'].apply(len) > 1]

We now create a list of unique permutations. We do not want users to have a relation to themselves, therefore we restrict it such that only pairs remain where the first and second element differ,

In [18]:
pairs = set([(a1, a2) for lst in retweeted_grouped['source_id_user'] for a1, a2 in itertools.permutations(lst, 2) if a1 != a2])

In [19]:
co_retweeted_df = pd.DataFrame(pairs, columns=["source_id", "target_id"])
co_retweeted_df["relation"] = "co_retweeted"
co_retweeted_df = co_retweeted_df[["source_id", "relation", "target_id"]]

In [20]:
co_retweeted_df.to_csv(OUT_DIR / "co_retweeted.csv", index=False)

# Co-Hashtag

To derive the *co-hashtag* relation we can follow a similar approach to *co-retweet*. We start out with:

| source_id | relation | target_id |
|----------------------|---------|----------|
| t1146699924525977611 | discuss | h4051 |
| t1296249717257625602 | discuss | h396644 |
| t1497194047244730369 | discuss | h5108039 |

And apply some transformation steps such that we end up with a table like this:

| source_id | relation | target_id |
|---------------------|---------|-------|
| u927875932093927424 | discuss | h4051 |
| u927875932093927424 | discuss | h25078 |
| u927875932093927424 | discuss | h2866 |

In [21]:
discuss_df = pd.read_csv(RELATION_DIR / 'discuss.csv')
discuss_df.shape

(66000633, 3)

In [22]:
by_hashtag = discuss_df.groupby("target_id")["source_id"].apply(list).reset_index()
by_hashtag.shape

(5146289, 2)

In [23]:
by_hashtag_user = pd.merge(discuss_df[["source_id", "target_id"]], post_df[["source_id", "target_id"]], left_on='source_id', right_on='target_id', suffixes=('_discuss', '_post'))
by_hashtag_user.shape

(66000633, 4)

In [24]:
by_hashtag_user = by_hashtag_user[["target_id_discuss", "source_id_post"]]
by_hashtag_user["relation"] = "discuss"
by_hashtag_user = by_hashtag_user[["source_id_post", "relation", "target_id_discuss"]]
by_hashtag_user.columns = ["source_id", "relation", "target_id"]

Drop duplicates, avoids the id twice inside `source_id` list.

In [25]:
by_hashtag_user = by_hashtag_user.drop_duplicates()

In [26]:
hashtags_grouped = by_hashtag_user.groupby("target_id")["source_id"].apply(list).reset_index()
hashtags_grouped = hashtags_grouped[hashtags_grouped['source_id'].apply(len) > 1]

Now we end up with a table like

| target_id |                                         source_id |
|-----------|---------------------------------------------------|
|        h0 | [u3159148494, u899737456408317952, u9083222, u... |
|        h1 | [u1058015808, u84757176, u1474033745707487239,... |
|      h100 | [u746639450, u722117494899679232, u12330805677... |

However, this has two problems, first of all some hashtags are overrepresentive, as we can see in the cell below:

In [27]:
hashtag_count = hashtags_grouped['source_id'].map(len)
hashtag_count.describe().map(lambda x: format(x, '.2f'))

count    1570584.00
mean          14.58
std          152.62
min            2.00
25%            2.00
50%            3.00
75%            7.00
max        82690.00
Name: source_id, dtype: object

As a consequence, we apply a filtering step by a threshold before we apply the next step.

In [28]:
hashtags_grouped['amount'] = hashtags_grouped['source_id'].apply(len)

In [29]:
PERCENT = .71  # THRESHOLD = 6.0
THRESHOLD = hashtags_grouped["amount"].quantile(PERCENT)
threshold_str = str(int(100*PERCENT))

In [30]:
mask = hashtags_grouped['amount'] <= THRESHOLD
hashtags_grouped_keep = hashtags_grouped[mask]

However, since the amount of hashtags we have is almost two orders of magnitude bigger than the amount of retweets, we can't just calculate all permutations as above.
Instead, we divide the DataFrame into smaller chunks, which we write to the disk.

In [31]:
batch_size = 100
for i in tqdm(range(math.ceil(hashtags_grouped_keep.shape[0] / batch_size))):
    source_ids = hashtags_grouped_keep.iloc[i*batch_size: (i+1)*batch_size]['source_id']

    co_hashtags = defaultdict(list)
    for row in source_ids:
        row_comb = [(a1, a2) for a1, a2 in itertools.permutations(row, 2)]
        [co_hashtags[a1].append(a2) for a1, a2 in row_comb]

    for user, matches in co_hashtags.items():
        filepath = OUT_DIR / 'tmp' / 'users' / user / f"co_hashtag_{threshold_str}.txt"
        filepath.parent.mkdir(exist_ok=True, parents=True)
        with open(filepath, 'a') as fh:
            fh.write('\n'.join(matches) + '\n')  # important + '\n'!

100%|██████████████████████████████████████████████████████████████████████████████████████| 11711/11711 [00:45<00:00, 256.71it/s]


Once all chunks are writte, we read `co_hashtag_*.txt` files into sub-DataFrames, which get then concatenated to one big DataFrame.

In [32]:
co_hashtag_df = pd.DataFrame([], columns=["source_id", "relation", "target_id", "amount"])

keys, vals = set(), set()
batch_size = 100
for i in tqdm(range(math.ceil(len(users) / batch_size))):    
    dfs = []
    for user in users[i*batch_size:(i+1)*batch_size]:
        filepath = OUT_DIR / 'tmp' / 'users' / user.name / f"co_hashtag_{threshold_str}.txt"
        if not filepath.is_file(): continue

        keys.add(user.name)
        
        with open(filepath, "r") as fh:
            target_ids = [user.rstrip() for user in fh.readlines()]

        for tid in target_ids:
            vals.add(tid)
        
        counter = Counter(target_ids)
        counter_df = pd.DataFrame(counter.items(), columns=["target_id", "amount"])
        counter_df["source_id"] = user.stem
        counter_df["relation"] = "co_hashtag"
        counter_df = counter_df[["source_id", "relation", "target_id", "amount"]]
        dfs.append(counter_df)
    X = pd.concat(dfs)
    partial_file = OUT_DIR / 'tmp' / 'partials' / threshold_str / f'cohashtag_partial_{threshold_str}_{i}.csv'
    partial_file.parent.mkdir(exist_ok=True, parents=True)
    X.to_csv(partial_file, index=False)    

100%|███████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [05:03<00:00, 32.90it/s]


Our final table is of this format:

| source_id | relation | target_id | amount |
|----------------------|------------|---------------------|---|
| u1470092268518395908 | co_hashtag | u3406899801 | 2 |
| u1113469923253211136 | co_hashtag | u793501498244292609 | 3 |
| u1113469923253211136 | co_hashtag | u314278227 | 2 |

In [33]:
partials = [file for file in (OUT_DIR / 'tmp' / 'partials' / threshold_str).rglob(f"*{threshold_str}_*.csv")]
partials_df = [pd.read_csv(f) for f in partials]
co_hashtag = pd.concat(partials_df)

In [34]:
co_hashtag.shape

(6954818, 4)

In [35]:
co_hashtag.to_csv(OUT_DIR / f"cohashtag_{threshold_str}.csv", index=False)

# Finalizing the csv for training

For our project we restrict ourselves to five types of relations:
- followers
- following
- retweeted_user
- co_retweeted
- co_hashtag

In [36]:
followers = pd.read_csv(RELATION_DIR / "followers.csv")
following = pd.read_csv(RELATION_DIR / "following.csv")

In [37]:
final = pd.concat([followers, following, retweeted_user, co_retweeted_df, co_hashtag])
final.shape

(12530639, 4)

In [38]:
final.to_csv(OUT_DIR / "combined" / "all_5rel.csv", index=False)